In [24]:
import json
from pathlib import Path

from openai import OpenAI
from pydantic import BaseModel

In [2]:
llm = OpenAI()

In [3]:
files = list(Path('../data').rglob('*.md'))

In [53]:
test = files[1]

In [54]:
with open(test, "r") as f:
    email = f.read()

In [68]:
with open("../data/reading_history.txt", "r") as f:
    reading_history = f.read()

with open("prompts/extraction.txt", "r") as f:
    prompt_extraction = f.read()

with open("prompts/infer_interests.txt", "r") as f:
    prompt_interest = f.read()

with open("prompts/personalization.txt", "r") as f:
    prompt_personalization = f.read()

In [56]:
class NewsletterItem(BaseModel):
    headline: str
    url: str
    description: str

class Newsletter(BaseModel):
    title: str
    items: list[NewsletterItem]

In [57]:
resp = llm.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt_extraction
        },
        {
            "role": "user",
            "content": email
        }
    ],
    response_format=Newsletter
)

In [58]:
json_string = resp.choices[0].message.content

In [59]:
resp = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt_interest
        },
        {
            "role": "user",
            "content": reading_history
        }
    ]
)

In [60]:
notes = resp.choices[0].message.content

In [61]:
personalization_input_format = f"""USER READING HISTORY: {reading_history}

USER INTEREST NOTES: {notes}

NEWSLETTER ITEMS: {json_string}
"""

In [69]:
personalization_resp = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt_personalization
        },
        {
            "role": "user",
            "content": personalization_input_format
        }
    ]
)

In [70]:
personalization_resp.choices[0].message.content

'```\n# Harris and Trump Squabble Over Debate Rules\n## Also, Russian missiles strike Ukraine’s energy infrastructure. Here’s the latest at the end of Monday.\n```'

In [67]:
print(personalization_input_format)

USER READING HISTORY: Harris Faces Challenge of Translating Convention Joy to Fall Momentum
U.S. Added 818,000 Fewer Jobs Than Reported Earlier
How Harris Has Completely Upended the Presidential Race, in 14 Maps
San Francisco Moves to Lead Fight Against Deepfake Nudes
Crucial Senate Race in Wisconsin Is Set Between Eric Hovde and Tammy Baldwin
Who Won the Olympics? All the Ways We Could Think Of to Track the Medal Count.
Plane Carrying 62 People Crashes in Brazil
Willing to Die for MrBeast (and $5 Million)
J.D. Vance Stumbles in His Debut, as Democrats Go on Offense
Harris Narrows Gap Against Trump, Times/Siena Poll Finds
Who Might Kamala Harris Pick as Her Running Mate?
Aaron Sorkin: How I Would Script This Moment for Biden and the Democrats
What Happened in the Final Moments as Biden Decided to Withdraw
Dozens of Local Police Officers Were at Trump's Rally. Very Few Were Watching a Critical Area.
Keanu Reeves Wrote a Book. A Really Weird One.
What Your Grocery Cart Says About You
Emp